In [1]:
import threading
import datetime

In [2]:

import pandas as pd
import numpy as np
import json
import requests
from math import radians, sqrt, sin, cos, atan2

dirpath = 'C:/Users/thuang1/Desktop/Snowplow/'
filename = '15min_test.csv'



In [3]:
# load data
test = pd.read_csv(dirpath+"15min_test.csv", header=0)

In [4]:
test.head(2)

,LABEL,XPOSITION,YPOSITION,HEADING,VELOCITY,ROADTEMP,AIRTEMP,SOLIDMATERIAL,LIQUIDMATERIAL,PREWETMATERIAL,...,UNDERBELLYPLOWSTATE,ODOMETER,SOLIDRUNTOTAL,LIQUIDRUNTOTAL,PREWETRUNTOTAL,SOLIDSETRATE,LIQUIDSETRATE,PREWETSETRATE,BLASTMODE,LOGDT_LOCAL
0,A33831,-92.101799,41.692169,276,96.304,NaN,NaN,0,0,0,...,0,NaN,50411,12693,0,0,0,0.0,NaN,20161211160000
1,A32592,-90.524384,41.807499,86,22.224,NaN,NaN,0,0,0,...,0,NaN,161976,67229,0,0,0,0.0,NaN,20161211160000


In [102]:
# split data into chuncks with 1500 records each. That's the limit on sending data to API.
n=1500

ans=[]
for i in range(int(len(test)/n)+1):
     ans.append(test[i*n:i*n+n])

## Rules in extract results from response.
- calculate the geometry distance (in feet).
- distinguish mainline and ramp.
- use the nearest routeId, if it is a ramp, check if there is any mainline within 20 ft. If so, use that mainline.

In [97]:
def extract_results(df):
    print ("thread started at: ", datetime.datetime.now())
    # get location
    d = []
    for i in range(len(df)):
        x = {'geometry':{'y':df.iloc[i]['YPOSITION'], 'x':df.iloc[i]['XPOSITION']}}
        d.append(x)

    locations = json.dumps(d, ensure_ascii=False)
    
    # get response
    url = 'https://gis.iowadot.gov/ramsa/rest/services/lrs/MapServer/exts/LRSServer/networkLayers/0/geometryToMeasure?'
    data = {'f':'json','locations': locations,'tolerance':'50','inSR':'4326'}
    response = requests.post(url, data=data)
    r=response.json()
    
    # extract results
    resall=[]
    for i in r['locations']:
        res=[]
        if i['status']==u'esriLocatingCannotFindLocation':
            routeid="0"
            measure=0
            x=0
            y=0
            res.append([routeid, measure, x, y])
        else:

            for j in i['results']:
                routeid=j['routeId'].strip('"\'')
                measure=j['measure']
                x=j['geometry']['x']
                y=j['geometry']['y']                
                res.append([routeid, measure, x, y])
        resall.append(res)

    datall=[]
    dat=[]

    for i in range(len(resall)):

        if resall[i][0][0]=='0':
            dat = [None, None]
        else:
            dat = get_routeId(d[i], resall[i])
        datall.append(dat)

    datall=np.array(datall)


    df['routeId']=datall[:,0]
    df['measure']=datall[:,1]
    
    print("thread complete at: ", datetime.datetime.now())

def geocalc(lat1, lon1, lat2, lon2):
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon1 - lon2

    EARTH_R = 6372.8*3280.84 # km to feet

    y = sqrt(
        (cos(lat2) * sin(dlon)) ** 2
        + (cos(lat1) * sin(lat2) - sin(lat1) * cos(lat2) * cos(dlon)) ** 2
        )
    x = sin(lat1) * sin(lat2) + cos(lat1) * cos(lat2) * cos(dlon)
    cc = atan2(y, x)
    return EARTH_R * cc


def get_routeId(d, dat):
    
            
    xlong = d['geometry']['x']
    xlati = d['geometry']['y']
    
    df2 = pd.DataFrame(dat, columns=['routeId', 'measure', 'long','lati'])
    
    df2['route_class']=df2.routeId.str[0]

    df2['ramp'] = 1
    df2.loc[(pd.isnull(df2.routeId.str[11])),'ramp'] = 0
    
    df2['dist']=df2.apply(lambda x: geocalc(float(x['lati']),float(x['long']),xlati,xlong), axis=1)
    
    df2['priority'] = 1 # low
    
    df2.loc[(df2.dist<=20) & (df2.ramp==0), 'priority'] = 0 # high
  
  
    df2 = df2.sort_values(by=['dist']).reset_index(drop=True)
    
    if df2.loc[0,'ramp']==1:
        df2 = df2.sort_values(by=['priority','dist']).reset_index(drop=True)
 
    routeId = df2.loc[0,'routeId']
    measure = df2.loc[0,'measure']

    return np.array([routeId, measure])







In [90]:
def thread_method(ans, i, n):
    c=0


    while c<n:

        ind=c+(i*2)

        t = threading.Thread(target=extract_results,args=(ans[ind],))
        t.start()
        c+=1


    while t.isAlive():
        print ('waiting')
        time.sleep(10)

    print ('All threads complete.')

In [103]:
# Main function

ind=0

for i in range(int(len(ans)/2)+1):
    print ('Round ', i, "started.")
    
    if len(ans)%2==1: # odd number of slice
        if i==int(len(ans)/2): # last round
            thread_method(ans, i, 1)
        else:
            thread_method(ans, i, 2)
    else:
        thread_method(ans, i, 2)
    



Round  0 started.
thread started at:  2017-11-01 13:03:16.421761
thread started at:  2017-11-01 13:03:16.430761
waiting
waiting
waiting
waiting
waiting
waiting
waiting
waiting
waiting
waiting
waiting
waiting


C:\Users\thuang1\AppData\Local\Continuum\Anaconda2\envs\Shuo\lib\site-packages\ipykernel\__main__.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\thuang1\AppData\Local\Continuum\Anaconda2\envs\Shuo\lib\site-packages\ipykernel\__main__.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


thread complete at:  2017-11-01 13:05:13.208561
All threads complete.
Round  1 started.
thread started at:  2017-11-01 13:05:16.565961
thread started at:  2017-11-01 13:05:16.597161
waiting
thread complete at:  2017-11-01 13:05:20.632561
waiting
waiting
waiting
waiting
waiting
waiting
waiting
waiting
waiting
waiting
waiting
thread complete at:  2017-11-01 13:07:14.294561
All threads complete.
Round  2 started.
thread started at:  2017-11-01 13:07:16.766161
thread started at:  2017-11-01 13:07:16.797361
waiting
thread complete at:  2017-11-01 13:07:21.623961
waiting
waiting
waiting
waiting
waiting
waiting
waiting
waiting
waiting
waiting
waiting
thread complete at:  2017-11-01 13:09:10.431361
thread complete at:  2017-11-01 13:09:12.958361
All threads complete.
Round  3 started.
thread started at:  2017-11-01 13:09:16.910961
thread started at:  2017-11-01 13:09:16.942161
waiting
waiting
waiting
waiting
waiting
waiting
waiting
waiting
waiting
waiting
waiting
waiting
waiting
thread complet

In [107]:
ans_backup = ans.copy() # back up your responses

## Rules in route correction:
- if a segment starts at route A, ends at route B, but its next segment starts at route B and returns to route A. Change the route B point by route A. Make them continuous on the route.
- ignore ramps (just keep them as R, do not check the detail routeId).
- remove all the segments with different start and end route. Because we do not know exactly when they made the turn.

In [128]:
# combine all the chuncks
cmb = ans[0].append(ans[1])
for i in range(2,len(ans)):
    cmb = cmb.append(ans[i])

In [129]:
cmb.tail(5)

,LABEL,XPOSITION,YPOSITION,HEADING,VELOCITY,ROADTEMP,AIRTEMP,SOLIDMATERIAL,LIQUIDMATERIAL,PREWETMATERIAL,...,SOLIDRUNTOTAL,LIQUIDRUNTOTAL,PREWETRUNTOTAL,SOLIDSETRATE,LIQUIDSETRATE,PREWETSETRATE,BLASTMODE,LOGDT_LOCAL,routeId,measure
13328,A33643,-91.556618,41.665691,280,1.852,NaN,NaN,0,0,0,...,49470,7571,0,0,0,0.0,NaN,20161211161559,S001920006W,66.3370298958
13329,A32896,-92.113876,41.692890,96,64.820,NaN,NaN,0,0,0,...,262346,40329,1634,0,0,0.0,NaN,20161211161559,S001910080E,214.168884756
13330,A34352,-93.383308,41.370960,270,42.596,NaN,NaN,0,0,0,...,24390,3284,0,0,0,0.0,NaN,20161211161559,S001930092W,140.697875442
13331,A33671,-91.626343,40.974232,215,55.560,NaN,NaN,0,0,0,...,7460,1431,72,0,0,0.0,NaN,20161211161559,S001920034W,35.4237093548
13332,A33847,-91.533340,40.939140,207,38.892,NaN,NaN,0,0,0,...,3759,736,3,0,0,0.0,NaN,20161211161559,S001930027S,248.61364428


### a. correct routes

In [130]:
def route_correction(a):
    
    a['route_correction']=0
    
    
    a['route_class']=a.routeId.str[0:10]
    a['END_ROUTECLASS']=a.END_ROUTEID.str[0:10] 
    a.loc[~pd.isnull(a.routeId.str[11]),'route_class']='R'
    a.loc[~pd.isnull(a.END_ROUTEID.str[11]),'END_ROUTECLASS']='R'
    a['diff']=0
    a.loc[a['END_ROUTECLASS']!=a['route_class'],'diff']=1
    a['NEXT_END']=a['END_ROUTECLASS'].shift(-1)
    a['NEXT_LABEL']=a['LABEL'].shift(-1)
    index = a.index[(a['diff']==1) & (a['route_class']==a['NEXT_END']) & (a['NEXT_LABEL']==a['LABEL'])]
    a.loc[index,'END_ROUTEID']=a.loc[index, 'routeId']
    a.loc[index+1,'routeId']=a.loc[index+1, 'END_ROUTEID']
    a.loc[index,'END_ROUTECLASS']=a.loc[index, 'route_class']
    a.loc[index+1,'route_class']=a.loc[index+1, 'END_ROUTECLASS']
    
    a.loc[index, 'route_correction']=2 # end point corrected
    a.loc[index+1, 'route_correction']=1 # start point corrected
    
    a = a.loc[a['route_class']==a['END_ROUTECLASS']]
    
    a=a.drop(['END_ROUTECLASS','route_class','NEXT_END','NEXT_LABEL','diff'],axis=1)
    return a

In [131]:
def route_process(a):
    a = a[pd.notnull(a['routeId'])]
    a = a.sort_values(by=['LABEL','LOGDT_LOCAL']).reset_index(drop=True)
    a['END_LABEL']=a['LABEL'].shift(-1)
    a['END_LATI']=a['YPOSITION'].shift(-1)
    a['END_LONG']=a['XPOSITION'].shift(-1)
    a['END_DT']=a['LOGDT_LOCAL'].shift(-1).apply(lambda x: '%.0f'%x)
    a['END_ROUTEID']=a['routeId'].shift(-1)
    a['END_MEASURE']=a['measure'].shift(-1)
    a.loc[a['LABEL']!=a['END_LABEL'],'END_LATI']=None
    a = a[pd.notnull(a['END_LATI'])]
    
    a = route_correction(a)
    
    return a

In [132]:
cmb = route_process(cmb)

### b. correct measures for those corrected routes

In [206]:
def measure_correction(df):
    

    corrected_point=df['route_correction']

    
    
    if (corrected_point==1):
        route='routeId'
        xx=df['XPOSITION']
        yy=df['YPOSITION']
    else:
        route='END_ROUTEID'
        xx=df['END_LONG']
        yy=df['END_LATI']
    
    get_measure_w_id(df, xx, yy, route)
    

def get_measure_w_id(df, xx, yy, route):
    
    d = []
    loc = {'geometry':{'y':yy, 'x':xx}, 'routeId':df[route]}
    d.append(loc)

    locations = json.dumps(d, ensure_ascii=False)
       

    url = 'https://gis.iowadot.gov/ramsa/rest/services/lrs/MapServer/exts/LRSServer/networkLayers/0/geometryToMeasure?'
    tolerance = 50
    
    data = {'f':'json','locations': locations,'tolerance':str(tolerance),'inSR':'4326'}
    response = requests.post(url, data=data)
    r=response.json()
    
    if r['locations'][0]['status']==u'esriLocatingCannotFindLocation':
        dat=None
    else:
        dat=(r['locations'][0]['results'][0]['measure'])

    
    return dat  

In [142]:
def measure_process(a):
    a.loc[a['route_correction']!=0,'corrected_measure']=a.loc[a['route_correction']!=0].apply(measure_correction, axis=1)
    a.loc[a['route_correction']==1,'measure']=a.loc[a['route_correction']==1,'corrected_measure']
    a.loc[a['route_correction']==2,'END_MEASURE']=a.loc[a['route_correction']==2,'corrected_measure']

    a = a.drop(['route_correction','corrected_measure'], axis=1)
    
    return a

In [136]:
cmb = measure_process(cmb)

In [146]:
cmb.loc[90:95]

,LABEL,XPOSITION,YPOSITION,HEADING,VELOCITY,ROADTEMP,AIRTEMP,SOLIDMATERIAL,LIQUIDMATERIAL,PREWETMATERIAL,...,BLASTMODE,LOGDT_LOCAL,routeId,measure,END_LABEL,END_LATI,END_LONG,END_DT,END_ROUTEID,END_MEASURE
91,A29870,-91.677307,41.298210,91,24.076,NaN,NaN,0,0,0,...,NaN,20161211160002,S001930092W,46.5075987889,A29870,41.298191,-91.676170,20161211160013,S001930092W,46.4484432461
92,A29870,-91.676170,41.298191,91,33.336,NaN,NaN,0,0,0,...,NaN,20161211160013,S001930092W,46.4484432461,A29870,41.298168,-91.674767,20161211160024,S001930092W,46.3754
93,A29870,-91.674767,41.298168,89,37.040,NaN,NaN,0,0,0,...,NaN,20161211160024,S001930092W,46.3754,A29870,41.298161,-91.673347,20161211160035,S001930092W,46.3015424648
94,A29870,-91.673347,41.298161,90,40.744,NaN,NaN,0,0,0,...,NaN,20161211160035,S001930092W,46.3015424648,A29870,41.298141,-91.671837,20161211160046,S001930092W,46.2229411759
95,A29870,-91.671837,41.298141,90,40.744,NaN,NaN,0,0,0,...,NaN,20161211160046,S001930092W,46.2229411759,A29870,41.298141,-91.670288,20161211160057,S001930092W,46.1423690656
